In [ ]:
#進度條 progress bar

class ProgressBar:
    bar_string_fmt = '\rProgress: [{}{}] {:.2%} {}/{}'
    cnt = 0

    def __init__(self, total, bar_total=20):
        self.total = total # task 的總數
        self.bar_total = bar_total  # progress bar 的長度

    def update(self, step=1):
        total = self.total
        self.cnt += step           # 更新 progress bar 的進度
        bar_cnt = (int((self.cnt/total)*self.bar_total))  # bar 的數量
        space_cnt = self.bar_total - bar_cnt  # 空白的數量

        # 顯示 progress bar '\r' 的意思代表 replace，print 出來的字串不會印在新的一行而是 replace 原本那行同位置的字符
        # {:.2%}，表示 format 進來的值會以百分比顯示，並只取到小數點後兩位
        progress = self.bar_string_fmt.format(
            '█' * bar_cnt,
            ' ' * space_cnt,
            self.cnt/total,
            self.cnt,
            total
        )
        print(progress, end='')

        # 100%
        if self.cnt/total == 1:
            print('\n')
            
if __name__ == '__main__':
    total = 10000
    progress_bar = ProgressBar(total)
    for _ in range(total):
        progress_bar.update()

In [ ]:
import json
import time
import requests
from pathlib import Path
from threading import Thread
from datetime import datetime
from fake_useragent import UserAgent

class DailyPriceSpider:
    def __init__(self, progress_bar=None):
        
        # 重複使用 TCP 連線
        self.req = requests.Session()
        self.url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY"
        self.headers = self.req.headers
        
        # 偽裝 User-Agent
        ua = UserAgent()
        self.headers["User-Agent"] = ua.random
        
        # 加入 progress bar
        if progress_bar:
            self.progress_bar = progress_bar
 
    # 供內部使用
    def __get(self, date, stock_no):
        res = self.req.get(self.url,
                           headers = self.headers,
                           params={
                               "response": "csv",  # 這次抓的是 csv 格式
                               "date": date,
                               "stockNo": stock_no
                           })
        return res.text
    
    # 供內部使用
    def __save_file(self, res_text, path):  # 去掉 res_text 裡多餘的空白行
        res_text = '\n'.join(
            filter(None,res_text.splitlines())
        )
        path = Path(path)
        
        # parents=True，如果父資料夾不存在則會一併創建
        # exist_ok=True，創建資料夾時，該資料夾已存在則不會 throw exception
        path.parent.mkdir(parents=True, exist_ok=True)
        with open(path, 'w', encoding="utf-8") as file:
            file.write(res_text)
        return
 
    
    def scrape(self, date, stock_no, save_path=""):
        # 把上面的兩個 function 整合
        res_text = self.__get(date, stock_no)
        if save_path:
            self.__save_file(res_text, save_path)
            
        # 更新 progress bar
        if self.progress_bar:
            self.progress_bar.update()


if __name__ == '__main__':
    SAVE_PATH_ROOT = "./daily_stock_price/"
    SLEEP_TIME = 3
    stock_info_list_file = {}
    with open("./stock_info_list.json", "r", encoding="utf-8") as f:
        stock_info_list_file = json.load(f)

    # 不管日期是幾號，他回傳都是給我們一整個月的，所以就固定設成1號就好
    # strftime()，把 datetime 輸出成我們要的格式
    today_date = "{}01".format(datetime.now().strftime("%Y%m"))

    stock_info_list = stock_info_list_file.get("stock", [])
    
    # 加入 progress bar
    progress_bar = ProgressBar(len(stock_info_list))
    dps = DailyPriceSpider(progress_bar=progress_bar)
    req_thread_list = []

    for stock_info in stock_info_list:

        stock_no = stock_info.get("stockNo")
        stock_name = stock_info.get("stockName")
        stock_industry = stock_info.get("stockIndustry")
        file_name = "{}_{}_daily_price.csv".format(
            today_date[:-2],  # 字串只需要用到年跟月
            stock_no+stock_name)

        save_path = "{}/{}/{}/{}".format(SAVE_PATH_ROOT,
                                         stock_industry,
                                         stock_no+stock_name,
                                         file_name)

        if stock_no and stock_name and stock_industry:
            req_thread = Thread(target=dps.scrape,
                                args=(today_date, stock_no, save_path),
                                daemon=True)
            req_thread.start()
            req_thread_list.append(req_thread)
            time.sleep(SLEEP_TIME)

    for req_thread in req_thread_list:
        req_thread.join()

    print("Finished")
